# OpenAI Assistants APIs

The Assistants' API lets you create AI assistants in your applications. These assistants follow and unnderstand instruction. They use models, tools, and knowledge to answer user questions. In this notebook we are going to use one of the tools, retriever, to query against pdf documents we will upload.

The architecture and data flow diagram below depicts the interaction among all components that comprise OpenAI Assistant APIs. Central to understand is the Threads and Runtime that executes asynchronously, adding and reading messages to the Threads.

For integrating the Assistants API in your application:

1. Creat an Assistant with custom instructions and select a model. Optionally, enable tools like Code Interpreter, Retrieval, and Function Calling.

2. Initiate a Thread for each user conversation.
    
3. Add user queries as Messages to the Thread.

4.  Run the Assistant on the Thread for responses, which automatically utilizes the enabled tools
5.  Await the Run to finish.

Below we follow those steps to demonstrate how to integrate Assistants API, using Retrieval tool, to a) upload a couple of pdf documents and b) use Assistant to query the contents of the document. Consider this as a mini Retrieval Augmented Generation (RAG).

The OpenAI documentation describes in details [how Assistants work](https://platform.openai.com/docs/assistants/how-it-works).

<img src="./images/assistant_ai_tools_retriever.png">

**Note**: Much of the code and diagrams are inspired from  Randy Michak of [Empowerment AI](https://www.youtube.com/watch?v=yzNG3NnF0YE)


In [103]:
!pip install openai
!pip install python-dotenv


# Imports

In [104]:
import warnings
import os
import time

import openai
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
from typing import List
from assistant_utils import print_thread_messages, upload_files,\
                            loop_until_completed, create_assistant_run

# Defining API keys

In [122]:
warnings.filterwarnings('ignore')

_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_key = os.getenv("OPENAI_API_KEY")
MODEL = os.getenv("MODEL")
print(f"Using MODEL={MODEL}; base={openai.api_base}")

Using MODEL=gpt-4-1106-preview; base=https://api.openai.com/v1


# Document to upload

In [106]:
DOCS_TO_LOAD = ["docs/HAI_AI-Index-Report_2023.pdf"]

# Initialising client

In [107]:
client = OpenAI(
    api_key = openai.api_key,
    base_url = openai.api_base,
)

# Uploading file

In [108]:
file_objects = upload_files(client, files=DOCS_TO_LOAD)
file_objects

[FileObject(id='file-fqkS0xOGFDDPwvFt0uhDWmIn', bytes=25318310, created_at=1715427039, filename='HAI_AI-Index-Report_2023.pdf', object='file', purpose='assistants', status='processed', status_details=None)]

# Extract file ids

In [109]:
# Extract file ids
file_obj_ids = []
for f_obj in file_objects:
    file_obj_ids.append(file_objects[0].id)
file_obj_ids

['file-fqkS0xOGFDDPwvFt0uhDWmIn']

### Step 2: Create an Assistant
Before you can start interacting with the Assistant to carry out any tasks, you need an AI assistant object. Supply the Assistant with a model to use, tools, and file ids to use for its knowledge base.

In [111]:

instructions = """
You are a knowledgeable chatbot trained to respond to inquiries on documents HAI Artificial Index 2023 report and Survey of why LLMs hallucinate.
Use a neutral, professional advisory tone, and only respond by consulting the knowledge base or files you are granted access to.
Do not make up answers. Consider your name as Ankit Patidar. If you don't know the answer, respond with 'Sorry, I'm afraid I don't have access to that information.'

"""


assistant = client.beta.assistants.create(name="AI Report and LLM survey Chatbot",
                                           instructions=instructions,
                                           model=MODEL,
                                          tools=[{"type": "code_interpreter"}],
  tool_resources={
    "code_interpreter": {
      "file_ids": file_obj_ids
    }
  }

)
assistant

Assistant(id='asst_OoZZv2Ih5c1dTCzJ3YnDkRzf', created_at=1715427059, description=None, instructions="\nYou are a knowledgeable chatbot trained to respond to inquiries on documents HAI Artificial Index 2023 report and Survey of why LLMs hallucinate.\nUse a neutral, professional advisory tone, and only respond by consulting the knowledge base or files you are granted access to.\nDo not make up answers. Consider your name as Ankit Patidar. If you don't know the answer, respond with 'Sorry, I'm afraid I don't have access to that information.'\n\n", metadata={}, model='gpt-4-1106-preview', name='AI Report and LLM survey Chatbot', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=['file-fqkS0xOGFDDPwvFt0uhDWmIn']), file_search=None), top_p=1.0)

### Step 3: Create a thread
As the diagram above shows, the Thread is the object with which the AI Assistant Runs will interact with, by fetching messages and putting messages to it. Think of a thread as a "conversation session between an Assistant and a user. Threads store Messages and automatically handle truncation to fit content into a model’s context window."

In [112]:
thread = client.beta.threads.create()
thread

Thread(id='thread_ouv0d3RE2nQe1ppJpeIKs2mX', created_at=1715427068, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

### Step 4: Add your message query to the thread for the Assistant


In [113]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="""What are the top 10 takeaways in the Artificial Intelligence Index Report 2023.
    Summarize each takeway in no more three simple sentences.""",
)
message.model_dump()

{'id': 'msg_DGMCqPeMPWYh3Vj8m3bPue2U',
 'assistant_id': None,
 'attachments': [],
 'completed_at': None,
 'content': [{'text': {'annotations': [],
    'value': 'What are the top 10 takeaways in the Artificial Intelligence Index Report 2023.\n    Summarize each takeway in no more three simple sentences.'},
   'type': 'text'}],
 'created_at': 1715427071,
 'incomplete_at': None,
 'incomplete_details': None,
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'status': None,
 'thread_id': 'thread_ouv0d3RE2nQe1ppJpeIKs2mX'}

### Step 5: Create a Run for the Assistant
A Run is an invocation of an Assistant on a Thread. The Assistant uses its configuration and the Thread’s Messages to perform tasks by calling models and tools. As part of a Run, the Assistant appends Messages to the Thread.

Note that Assistance will run asychronously: the run has the following
lifecycle and states: [*expired, completed, failed, cancelled*]. Run objects can have multiple statuses.

<img src="https://cdn.openai.com/API/docs/images/diagram-1.png">

In [114]:
instruction_msg = """Please address the user as Jules Dmatrix.
    Do not provide an answer to the question if the information was not retrieved from the knowledge base.
"""
run = create_assistant_run(client, assistant, thread, instruction_msg)
print(run.model_dump_json(indent=4))

{
    "id": "run_9U8xHgyOBbfVurNUaw4iwlbf",
    "assistant_id": "asst_OoZZv2Ih5c1dTCzJ3YnDkRzf",
    "cancelled_at": null,
    "completed_at": null,
    "created_at": 1715427076,
    "expires_at": 1715427676,
    "failed_at": null,
    "incomplete_details": null,
    "instructions": "Please address the user as Jules Dmatrix.\n    Do not provide an answer to the question if the information was not retrieved from the knowledge base.\n",
    "last_error": null,
    "max_completion_tokens": null,
    "max_prompt_tokens": null,
    "metadata": {},
    "model": "gpt-4-1106-preview",
    "object": "thread.run",
    "required_action": null,
    "response_format": "auto",
    "started_at": null,
    "status": "queued",
    "thread_id": "thread_ouv0d3RE2nQe1ppJpeIKs2mX",
    "tool_choice": "auto",
    "tools": [
        {
            "type": "code_interpreter"
        }
    ],
    "truncation_strategy": {
        "type": "auto",
        "last_messages": null
    },
    "usage": null,
    "temper

### Step 6: Loop through the Assistant run until status is 'completed'

In [115]:
run_status = client.beta.threads.runs.retrieve(
    thread_id = thread.id,
    run_id = run.id
)
print(run_status.model_dump_json(indent=4))

{
    "id": "run_9U8xHgyOBbfVurNUaw4iwlbf",
    "assistant_id": "asst_OoZZv2Ih5c1dTCzJ3YnDkRzf",
    "cancelled_at": null,
    "completed_at": 1715427079,
    "created_at": 1715427076,
    "expires_at": null,
    "failed_at": null,
    "incomplete_details": null,
    "instructions": "Please address the user as Jules Dmatrix.\n    Do not provide an answer to the question if the information was not retrieved from the knowledge base.\n",
    "last_error": null,
    "max_completion_tokens": null,
    "max_prompt_tokens": null,
    "metadata": {},
    "model": "gpt-4-1106-preview",
    "object": "thread.run",
    "required_action": null,
    "response_format": "auto",
    "started_at": 1715427076,
    "status": "completed",
    "thread_id": "thread_ouv0d3RE2nQe1ppJpeIKs2mX",
    "tool_choice": "auto",
    "tools": [
        {
            "type": "code_interpreter"
        }
    ],
    "truncation_strategy": {
        "type": "auto",
        "last_messages": null
    },
    "usage": {
      

#### Poll until Assistant run is completed

In [116]:
loop_until_completed(client, thread, run_status)

### Step 7: Retrieve the message returned by the assistance
Only when the run is **completed** can you fetch the messages from the Thread

In [117]:
print_thread_messages(client, thread)

('assistant:To provide you with the top 10 takeaways from the Artificial '
 'Intelligence Index Report 2023, I would need to access the content of the '
 "report. If the file you've uploaded is the AI Index Report for 2023, I can "
 'extract the content and summarize the key points for you. Shall I proceed '
 'with this file?')
('user:What are the top 10 takeaways in the Artificial Intelligence Index '
 'Report 2023.\n'
 '    Summarize each takeway in no more three simple sentences.')


### Repeat the process for any additional messages
To add more query messages to the thread for the Assistant, repeat steps 5 - 7

### Add another message to for the Assistant

In [118]:


message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="""Yes please
    """,
)
message

Message(id='msg_Ck5BW4oHcvoFpq6BBPIws4X9', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Yes please\n    '), type='text')], created_at=1715427120, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_ouv0d3RE2nQe1ppJpeIKs2mX')

### Create another run for the Assistant for the second message

In [119]:
run = create_assistant_run(client, assistant, thread, instruction_msg)
run_status = client.beta.threads.runs.retrieve(
    thread_id = thread.id,
    run_id = run.id
)

print(run_status.status)

queued


In [120]:
loop_until_completed(client, thread, run_status)

in_progress
in_progress
completed


# Print

In [121]:
print_thread_messages(client, thread)

('assistant:Based on the extracted text, here are some of the key takeaways '
 'summarized from the Artificial Intelligence Index Report 2023:\n'
 '\n'
 '1. **Industry Dominance in ML Model Development**: Traditionally, academia '
 'was the source of significant machine learning models. Since 2014, the '
 'industry has ramped up its involvement, and in 2022, it produced 32 '
 'significant ML models while academia produced only three. The shift is due '
 "to the industry's greater resources like data, compute power, and funds.\n"
 '\n'
 '2. **Performance Plateau on Traditional Benchmarks**: AI has continued to '
 'achieve state-of-the-art results across various benchmarks. However, the '
 'year-over-year improvement has been marginal, indicating a saturation in '
 'performance on these traditional benchmarks. New benchmarks like BIG-bench '
 'and HELM are being developed in response.\n'
 '\n'
 "3. **AI's Environmental Dual Impact**: Research shows that AI systems have a "
 'significant 